# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import re
import pickle
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import warnings
warnings.filterwarnings("ignore")

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import TruncatedSVD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql_table('Messages',engine)
X = df.message.values
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
# Write tokenization function to process text data
def tokenize(text):
    
    # Normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())

    # Token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]

    # Lemmatizer and clean
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Build machine learning pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Evaluate metrics of ML pipeline 
def eval_metrics(ArrayL, ArrayP, col_names):
    
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics

In [9]:
# Calculate evaluation metrics for training set
y_train_pred = pipeline.predict(X_train)
col_names = list(y.columns.values)

print(eval_metrics(np.array(y_train), y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.989755   0.991883  0.994795  0.993337
request                 0.987706   0.994088  0.935304  0.963801
offer                   0.999129   1.000000  0.808989  0.894410
aid_related             0.985042   0.995600  0.968570  0.981899
medical_help            0.987091   0.997729  0.841098  0.912742
medical_products        0.990830   0.997546  0.821212  0.900831
search_and_rescue       0.995338   1.000000  0.828302  0.906089
security                0.995748   1.000000  0.764873  0.866774
military                0.995236   1.000000  0.861607  0.925659
water                   0.995185   0.999133  0.925301  0.960801
food                    0.995338   0.997618  0.960550  0.978733
shelter                 0.992623   0.999396  0.920511  0.958333
clothing                0.998207   0.992647  0.891089  0.939130
money                   0.996107   0.997175  0.824766  0.902813
missing_people          0.996978   1.000

In [10]:
# Calculate evaluation metrics for test set
y_test_pred = pipeline.predict(X_test)

eval_metrics0 = eval_metrics(np.array(y_test), y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.802828   0.838830  0.915041  0.875279
request                 0.889196   0.761240  0.464083  0.576629
offer                   0.995543   0.000000  0.000000  0.000000
aid_related             0.750884   0.745152  0.601565  0.665704
medical_help            0.920393   0.495050  0.096712  0.161812
medical_products        0.953127   0.714286  0.092879  0.164384
search_and_rescue       0.970647   0.588235  0.051546  0.094787
security                0.981405   0.000000  0.000000  0.000000
military                0.972337   0.600000  0.127660  0.210526
water                   0.952205   0.858025  0.325527  0.471986
food                    0.931919   0.815126  0.522207  0.636587
shelter                 0.936530   0.740566  0.304854  0.431912
clothing                0.986783   0.863636  0.186275  0.306452
money                   0.973874   1.000000  0.034091  0.065934
missing_people          0.988935   0.000

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# Review params to get ideas to improve model
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f94da3d0400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
# Perform grid search to find better parameters
parameters = {'tfidf__use_idf': (True, False),
              'vect__ngram_range': ((1, 1), (1, 2)),
              'clf__estimator__n_estimators': [10,20],
}
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3)

# Find best parameters
T_model = cv.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.22702591947546358, total= 2.0min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.23524590163934425, total= 2.0min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.23184099989755147, total= 2.4min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.23063524590163934, total= 2.4min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.22948468394631696, total= 2.0min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.23237704918032787, total= 2.0min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 2), score=0.22938223542669808, total= 2.5min
[CV] clf__estimator

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 55.2min finished


In [13]:
# Grid search results
T_model.cv_results_

{'mean_fit_time': array([  69.11949754,   94.43226147,   70.08173239,   97.21860027,
          90.94400918,  149.30103743,   93.33645368,  151.88576055]),
 'std_fit_time': array([ 0.69949532,  0.57339287,  1.01761878,  0.00297737,  0.2424165 ,
         1.02877223,  0.20926046,  0.86911058]),
 'mean_score_time': array([ 49.76156354,  50.82567406,  50.49769127,  51.38450336,
         52.66521215,  55.63835096,  52.94630396,  55.47585034]),
 'std_score_time': array([ 1.16416526,  1.09567046,  0.69959939,  1.04526377,  1.41914487,
         0.26248193,  1.48756683,  1.53438902]),
 'param_clf__estimator__n_estimators': masked_array(data = [10 10 10 10 20 20 20 20],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True True False False True True False False],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__ngram_range': masked_array(data 

In [14]:
# Top mean score
np.max(T_model.cv_results_['mean_test_score'])

0.24394242098253163

In [15]:
# Parameters for top mean score
T_model.best_params_

{'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': False,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Evaluate metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.814661   0.837636  0.936382  0.884261
request                 0.898417   0.816587  0.483932  0.607715
offer                   0.995543   0.000000  0.000000  0.000000
aid_related             0.764561   0.778423  0.599702  0.677474
medical_help            0.921777   0.547619  0.088975  0.153078
medical_products        0.953281   0.771429  0.083591  0.150838
search_and_rescue       0.970647   0.615385  0.041237  0.077295
security                0.981866   0.500000  0.008475  0.016667
military                0.971108   0.500000  0.063830  0.113208
water                   0.955433   0.870270  0.377049  0.526144
food                    0.932227   0.881313  0.469717  0.612818
shelter                 0.935761   0.793939  0.254369  0.385294
clothing                0.985861   0.812500  0.127451  0.220339
money                   0.973567   1.000000  0.022727  0.044444
missing_people          0.989089   0.000

In [17]:
# Summary of first model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943972,0.516034,0.197523,0.253236
std,0.056416,0.353965,0.248534,0.272337
min,0.750884,0.000000,0.000000,0.000000
25%,0.937221,0.035714,0.001276,0.002463
50%,0.957584,0.666667,0.092879,0.161812
75%,0.981481,0.790279,0.320745,0.457828
max,0.995543,1.000000,0.915041,0.875279


In [18]:
# Summary of tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.945224,0.580912,0.185548,0.237109
std,0.053560,0.337445,0.253397,0.279496
min,0.764561,0.000000,0.000000,0.000000
25%,0.937145,0.416667,0.008270,0.016207
50%,0.958353,0.729167,0.044643,0.083770
75%,0.981328,0.827112,0.355191,0.495885
max,0.995543,1.000000,0.936382,0.884261


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# Improve the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [20]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline 2
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [21]:
# Calculate evaluation metrics for training set 2
y_train_pred2 = pipeline2.predict(X_train)
col_names_ = list(y.columns.values)

print(eval_metrics(np.array(y_train), y_train_pred2, col_names_))

                        Accuracy  Precision    Recall        F1
related                 0.764561   0.764646  0.999464  0.866426
request                 0.826392   1.000000  0.001179  0.002355
offer                   0.996107   1.000000  0.025641  0.050000
aid_related             0.582859   0.575540  0.009799  0.019270
medical_help            0.919062   1.000000  0.001264  0.002525
medical_products        0.948978   1.000000  0.001003  0.002004
search_and_rescue       0.972030   1.000000  0.005464  0.010870
security                0.982019   1.000000  0.002841  0.005666
military                0.966959   1.000000  0.001548  0.003091
water                   0.935710   1.000000  0.000796  0.001591
food                    0.886379   1.000000  0.000901  0.001800
shelter                 0.910609   1.000000  0.003427  0.006830
clothing                0.983915   1.000000  0.006329  0.012579
money                   0.976538   1.000000  0.004348  0.008658
missing_people          0.989242   1.000

In [22]:
# Calculate evaluation metrics for test set 2
y_test_pred2 = pipeline2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(y_test), y_test_pred2, col_names)
print(eval_metrics2)

                        Accuracy  Precision    Recall        F1
related                 0.768403   0.768697  0.998598  0.868694
request                 0.833871   0.000000  0.000000  0.000000
offer                   0.993853   0.000000  0.000000  0.000000
aid_related             0.585062   0.454545  0.007418  0.014599
medical_help            0.922852   0.000000  0.000000  0.000000
medical_products        0.951283   0.000000  0.000000  0.000000
search_and_rescue       0.972645   0.000000  0.000000  0.000000
security                0.981405   0.000000  0.000000  0.000000
military                0.967112   0.000000  0.000000  0.000000
water                   0.936069   0.000000  0.000000  0.000000
food                    0.891963   0.000000  0.000000  0.000000
shelter                 0.913478   0.000000  0.000000  0.000000
clothing                0.986322   0.000000  0.000000  0.000000
money                   0.977870   0.000000  0.000000  0.000000
missing_people          0.986476   0.000

In [23]:
# Look at parameters for improving model
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('best', TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
          random_state=None, tol=0.0)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
    

In [24]:
# Model tuning with added parameters
parameters2 = {'tfidf__use_idf': (True, False), 
               'clf__estimator__n_estimators': [50, 100],
               'clf__estimator__learning_rate': [1,2]
              }
cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters2, cv=2, n_jobs=-1, verbose=3)

# Find best parameters
T_model2 = cv2.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.18676365126523922, total=  15.1s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   17.5s remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.18493852459016394, total=  16.0s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   35.9s remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.18010449749001126, total=  14.8s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.18370901639344261, total=  15.1s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.18276815900010246, total=  29.3s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.18299180327868853, total=  29.4s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  9.7min finished


In [25]:
# Top mean score for model 2
np.max(T_model2.cv_results_['mean_test_score'])

0.1858511346754777

In [26]:
# Parameters for top mean score model 2
T_model2.best_params_

{'clf__estimator__learning_rate': 1,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True}

### Testing Model

In [27]:
# Evaluating metrics for test set model 2
tuned_pred_test2 = T_model2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

                        Accuracy  Precision    Recall        F1
related                 0.768403   0.768697  0.998598  0.868694
request                 0.833871   0.000000  0.000000  0.000000
offer                   0.993853   0.000000  0.000000  0.000000
aid_related             0.583525   0.464286  0.033754  0.062932
medical_help            0.922852   0.000000  0.000000  0.000000
medical_products        0.951283   0.000000  0.000000  0.000000
search_and_rescue       0.972645   0.000000  0.000000  0.000000
security                0.981405   0.000000  0.000000  0.000000
military                0.967112   0.000000  0.000000  0.000000
water                   0.936069   0.000000  0.000000  0.000000
food                    0.891963   0.000000  0.000000  0.000000
shelter                 0.913478   0.000000  0.000000  0.000000
clothing                0.986169   0.000000  0.000000  0.000000
money                   0.977870   0.000000  0.000000  0.000000
missing_people          0.986476   0.000

In [28]:
# Summary of original model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943972,0.516034,0.197523,0.253236
std,0.056416,0.353965,0.248534,0.272337
min,0.750884,0.000000,0.000000,0.000000
25%,0.937221,0.035714,0.001276,0.002463
50%,0.957584,0.666667,0.092879,0.161812
75%,0.981481,0.790279,0.320745,0.457828
max,0.995543,1.000000,0.915041,0.875279


In [29]:
# Summary of tuned model 1
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.945224,0.580912,0.185548,0.237109
std,0.053560,0.337445,0.253397,0.279496
min,0.764561,0.000000,0.000000,0.000000
25%,0.937145,0.416667,0.008270,0.016207
50%,0.958353,0.729167,0.044643,0.083770
75%,0.981328,0.827112,0.355191,0.495885
max,0.995543,1.000000,0.936382,0.884261


In [30]:
# Summary of tuned model 2
eval_metrics2.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.924617,0.052688,0.029627,0.026879
std,0.088879,0.169049,0.168700,0.146865
min,0.583525,0.000000,0.000000,0.000000
25%,0.909636,0.000000,0.000000,0.000000
50%,0.951283,0.000000,0.000000,0.000000
75%,0.980867,0.000000,0.000000,0.000000
max,0.995236,0.768697,0.998598,0.868694


### Evaluating Tuned Models

* Model 1 has higher mean in all categories (accuracy, precision, recall, F1 value). The standard deviation for model 1 is higher, indicating a greater distribution in relation to the mean values
* Model 2 has lower mean value for all categories and smaller standard deviation for each metric when compared to model 1

* Comparing metrics (mean value) for model 1 and model 2
>* **Accuracy**: Both model 1 and model 2 have pretty high accuracy (0.942185 and 0.925368, respectively)
>* **Precision**: Model 1 has a higher average precision of 0.559671 when compared to model 2 (0.081468). The higher precision of model 1, indicates that this model is better at predicting actual positives and is better for predicting when the costs of a false positive is high.
>* **Recall**: Model 1 has a higer recall of 0.171154 when compared to model 2's recall of 0.030304. The higher recall value in model 1, calculates how many of the actual positives the model captures that are a true positive. This means model 1 would be best selected when there's a high cost associated with false negatives. 
>* **F1**: Model 1 has a higher F1 value of 0.225254 when compared to model 2 (0.028043). The F1 score is best used if we're seeking to find a balance between Precision and Recall and if there's an uneven distribution. An F1 score of 1 is considered a 'perfect' model, while and F1 score of 0 indicates a bad model. The higher F1 value for model 1 is indicative that the model has low false positives and low false negatives, so it's more effective at correctly identifying real threats and isn't disturbed by false alarms. 

* Based on the metrics, Model 1 is the best overall model and will be saved for continuation of this work

### 9. Export your model as a pickle file

In [31]:
# Export best model as pickle file
pickle.dump(T_model, open('classifer.pkl', 'wb'))

### Sources
* https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
* https://pathmind.com/wiki/accuracy-precision-recall-f1#:~:text=That%20is%2C%20a%20good%20F1,total%20failure%20when%20it's%200%20
* https://apapiu.github.io/2016-08-04-tf_idf/
* https://www.greenbook.org/marketing-research/how-to-interpret-standard-deviation-and-standard-error-in-survey-research-03377
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html